## Základní připojení k API

### Zkopírujte si tento notebook do svého Google Drive

### Nastavte si API klíč

In [3]:
try:
    from google.colab import userdata
    _secret = userdata.get("GOOGLE_API_KEY")
    !pip install instructor google-generativeai python-dotenv
except ImportError:
    import os
    from dotenv import load_dotenv
    _ = load_dotenv()
    _secret = os.environ.get("GOOGLE_API_KEY")

In [ ]:
import instructor
import google.generativeai as genai
import os

try:
    genai.configure(api_key=_secret)
except KeyError:
    print("Nastavte si API klíč v proměnné GOOGLE_API_KEY a znovu spusťte notebook")

client = instructor.from_gemini(
    client=genai.GenerativeModel(
        model_name="models/gemini-1.5-flash-latest",
    ),
    mode=instructor.Mode.GEMINI_JSON,
)
test = client.chat.completions.create(
    messages=[
        {"role": "user", "content": "Hello, how are you?"},
    ],
    response_model=str,
)
print(test)
